In [ ]:
def get_securities_codes(start_page, end_page):
    url='https://minkabu.jp/financial_item_ranking/dividend_yield?page={}'
    dd_list=[]
    for i in range(start_page, end_page+1):
        target_url=url.format(i)
        r=requests.get(target_url)
        sleep(5)
        soup=BeautifulSoup(r.text,'html.parser')
        tab=soup.find('table',class_='md_table rnk_li')
        conts=tab.find_all('tr')
        del conts[0]

        for cont in conts:
            rank=cont.find('span',class_='ranking_no fsize_l').text
            code=cont.find('div',class_='md_sub').text
            name=cont.find('div',class_='fwb w90p').text
            y=cont.find('td',class_='tar cur vamd').text
            yeild= y.strip().replace('%', '')
            dic={
                'ランキング':rank,
                '証券コード':code,
                '企業名':name,
                '配当利回り':yeild,
            }
            if float(yeild) >= 3.75:
                dd_list.append(dic)
    securities_codes = [d['証券コード'] for d in dd_list]
    return securities_codes


In [ ]:
securities_codes = get_securities_codes(36, 37)

# securities_codes の中身を確認する
print(securities_codes)

# securities_codes を使用する処理をここに追加


In [ ]:
def sales_chenge_int(sales):
    sales_list=[sale.text for sale in sales]
    eval_sale_list = []
    for sale in sales_list:
        # 兆しかない時
        if '兆' in sale and '億' not in sale and '万' not in sale:
            hennkan = sale.replace('-1', '*10**12')
            eval_sale_list.append(eval(hennkan))
        # 億しかない時
        elif '億' in sale and '兆' not in sale and '万' not in sale:
            hennkan = sale.replace('億', '*10**8')
            eval_sale_list.append(eval(hennkan))
        # 万しかない時
        elif '万' in sale and '兆' not in sale and '万' not in sale:
            hennkan = sale.replace('万', '*10**4')
            eval_sale_list.append(eval(hennkan))
        # 兆と億の時
        elif'兆'in sale and '億' in sale:
            hennkan = sale.replace('兆', '*10**12+').replace('億','*10**8')
            eval_sale_list.append(eval(hennkan))
        # 億と万の時
        elif'億'in sale and '万' in sale:
            hennkan = sale.replace('億', '*10**8+').replace('万','*10**4')
            eval_sale_list.append(eval(hennkan))

        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sale)
            eval_sale_list.append(eval(hennkan))

    return eval_sale_list


In [3]:
def check_sales_list(eval_sale_list):
    result = all(int(eval_sale_list[i]) < int(eval_sale_list[i+1]) for i in range(len(eval_sale_list)-1))
    uriagedaka_stock=[]
    if result:
        uriagedaka_stock.append('OK')
    else:
        for i in range(len(eval_sale_list)-1):
            if int(eval_sale_list[i]) >= int(eval_sale_list[i+1]):
                uriagedaka_stock.append(f'NOT (at index {i+1})')
    return uriagedaka_stock

def check_eps(zeps_list):
    result = all(int(zeps_list[i]) < int(zeps_list[i+1]) for i in range(len(zeps_list)-1))
    eps_stock = []
    if result:
        eps_stock.append('OK')
    else:
        for i in range(len(zeps_list)-1):
            if int(zeps_list[i]) >= int(zeps_list[i+1]):
                eps_stock.append(f'NOT (at index {i+1})')
    return eps_stock

def check_eiri(zop_list):
    zop_list = [float(x) for x in zop_list]
    resault = all(10 <= float(zop_list[i]) for i in range(len(zop_list)))
    eiri_stock = []
    if resault:
        eiri_stock.append('OK')
    else:
        for i in range(len(zop_list)):
            if 40 >= float(zop_list[i]):
                eiri_stock.append(f'NOT (at index {i})')
    return eiri_stock


def check_zikosihon(zoption_list):
    oolist = [float(x) for x in zoption_list]
    result = all(40 <= float(oolist[i]) for i in range(len(oolist)))
    zikosihon_stock = []
    if result:
        zikosihon_stock.append('OK')
    else:
        for i in range(len(oolist)):
            if 40 >= float(oolist[i]):
                zikosihon_stock.append(f'NOT (at index {i})')
    return zikosihon_stock

def check_genkin(eval_cash_list):
    result = all(int(eval_cash_list[i]) < int(eval_cash_list[i+1]) for i in range(len(eval_cash_list)-1))
    genkin_stock = []
    if result:
        genkin_stock.append('OK')
    else:
        for i in range(len(eval_cash_list)-1):
            if int(eval_cash_list[i]) >= int(eval_cash_list[i+1]):
                genkin_stock.append(f'NOT (at index {i+1})')
    return genkin_stock


def check_cflow_order(eval_cf_flow_list):
　　　　result = all(int(eval_cf_flow_list[i]) < int(eval_cf_flow_list[i+1]) for i in range(len(eval_cf_flow_list)-1))
  　　cflow_stock=[]
  　　if result:
      　　cflow_stock.append('OK')
  　　else:
      　　for i in range(len(eval_cf_flow_list)-1):
          　　if int(eval_cf_flow_list[i]) >= int(eval_cf_flow_list[i+1]):
              　　cflow_stock.append(f'NOT (at index {i+1})')
  　　return cflow_stock

def check_zstock_order(zstock_list):
  　　result = all(float(zstock_list[i]) < float(zstock_list[i+1]) for i in range(len(zstock_list)-1))
  　　hitokabu_stock=[]
  　　if result:
      　　hitokabu_stock.append('OK')
  　　else:
      　　for i in range(len(zstock_list)-1):
          　　if float(zstock_list[i]) >= float(zstock_list[i+1]):
              　　hitokabu_stock.append(f'NOT (at index {i+1})')
  　　return hitokabu_stock

def check_zreturn_range(zreturn_list):
    int_zreturn_list = [float(x) for x in zreturn_list]
    result = all(30<int(int_zreturn_list[i]<50) for i in range(len(int_zreturn_list)))
    seikou_stock=[]
    if result:
        seikou_stock.append('OK')
    else:
        for i in range(len(int_zreturn_list)):
            if int(30>int_zreturn_list[i] or int_zreturn_list[i]>70):
                seikou_stock.append(f'NOT (at index {i})')
    return seikou_stock




def get_closing_url(code):
    url = f'https://irbank.net/{code}/ir'
    r = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(r.text,'html.parser')
    element = soup.select_one('div.csb.cc2 > ul:nth-of-type(2) > li:first-of-type')
    taget = element.find('a')
    link = taget.get('href')
    return link


SyntaxError: invalid non-printable character U+3000 (1453500064.py, line 61)

In [6]:
        def convert_sales_figure(sales_figure: str) -> int:
            #兆しかない時
            if '兆' in sales_figure and '億' not in sales_figure and '万' not in sales_figure:
                hennkan = sales_figure.replace('兆', '*10**12')
                return eval(hennkan)
            #億しかない時
            elif '億' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
                hennkan = sales_figure.replace('億', '*10**8')
                return eval(hennkan)
            #万しかない時
            elif '万' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
                hennkan = sales_figure.replace('万', '*10**4')
                return eval(hennkan)
            #兆と億の時
            elif'兆'in sales_figure and '億' in sales_figure:
                hennkan = sales_figure.replace('兆', '*10**12+').replace('億','*10**8')
                return eval(hennkan)
            #億と万の時
            elif'億'in sales_figure and '万' in sales_figure:
                hennkan = sales_figure.replace('億', '*10**8+').replace('万','*10**4')
                return eval(hennkan)
            else:
                hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sales_figure)
                return eval(hennkan)




In [ ]:
eval_sale_list = []
        for sale in sales_list:
            eval_sale_list.append(convert_sales_figure(sale))

In [4]:
def convert_eps(eps_figure: str) -> int:
    number_str = eps_figure.split('円')[0]
    hennkan = number_str + '*100'
    return eval(hennkan)


In [ ]:
 zeps_list = []
            for ep in eps_list:
                zeps_list.append(convert_eps(ep))


In [10]:
def check_uriagedaka(eval_sale_list):
    result = all(int(eval_sale_list[i]) < int(eval_sale_list[i+1]) for i in range(len(eval_sale_list)-1))
    uriagedaka_stock = []
    if result:
        uriagedaka_stock.append('OK')
    else:
        for i in range(len(eval_sale_list)-1):
            if int(eval_sale_list[i]) >= int(eval_sale_list[i+1]):
                uriagedaka_stock.append(f'NOT (at index {i+1})')
    return uriagedaka_stock

 def check_sales_list(zeps_list):
            result = all(int(zeps_list[i]) < int(zeps_list[i+1]) for i in range(len(zeps_list)-1))
            eps_stock=[]
            if result:
                eps_stock.append('OK')
            else:
                for i in range(len(eps_stock)-1):
                    if int(eps_stock[i]) >= int(eps_stock[i+1]):
                        eps_stock.append(f'NOT (at index {i+1})')
            return eps_stock

In [ ]:

def process_cashes(cashes):
    cash_list = [cash.text for cash in cashes]
    eval_cash_list = []
    for cash in cash_list:
        # 兆しかない時
        if '兆' in cash and '億' not in cash and '万' not in cash:
            hennkan = cash.replace('兆', '*10**12')
            eval_cash_list.append(eval(hennkan))
        # 億しかない時
        elif '億' in cash and '兆' not in cash and '万' not in cash:
            hennkan = cash.replace('億', '*10**8')
            eval_cash_list.append(eval(hennkan))
        # 万しかない時
        elif '万' in cash and '兆' not in cash and '万' not in cash:
            hennkan = cash.replace('万', '*10**4')
            eval_cash_list.append(eval(hennkan))
        # 兆と億の時
        elif '兆' in cash and '億' in cash:
            hennkan = cash.replace('兆', '*10**12+').replace('億','*10**8')
            eval_cash_list.append(eval(hennkan))
        # 億と万の時
        elif '億' in cash and '万' in cash:
            hennkan = cash.replace('億', '*10**8+').replace('万','*10**4')
            eval_cash_list.append(eval(hennkan))
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sale)
            eval_cash_list.append(eval(hennkan))
    return eval_cash_list


In [ ]:
def convert_stock(stock_price: str) -> str:
    if '.' in stock_price:
        number_str = stock_price.split('円')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    elif '*' in stock_price:
        kn = stock_price.split('*')[1]
        number_str1 = kn .split('円')[0]
        integer_part1 = number_str1.zfill(2)
        return f"{integer_part1}.00"
    else:
        number_str = stock_price.split('円')[0]
        integer_part = number_str.zfill(2)
        return f"{integer_part}.00"


In [ ]:
def convert_return(return_figure: str) -> str:
    if '.' in return_figure:
        number_str = return_figure.split('%')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    else:
        integer_part = return_figure.split('%')[0]
        integer_part = integer_part.zfill(2)
        return f"{integer_part}.00"


In [ ]:
def convert_japanese_financial_string(string):
    hennkan = string.replace('兆', '*10**12').replace('億', '*10**8').replace('万', '*10**4')
    return eval(hennkan)




In [ ]:
eval_cf_flow_list = [convert_japanese_financial_string(flow) for flow in cf_flow_list]

In [7]:
        def convert_flow_figure(flow_figure: str) -> int:
                #兆しかない時
                if '兆' in flow_figure and '億' not in flow_figure and '万' not in flow_figure:
                    hennkan = flow_figure.replace('兆', '*10**12')
                    return eval(hennkan)         
                #億しかない時
                elif '億' in flow_figure and '兆' not in flow_figure and '万' not in flow_figure:
                    hennkan = flow_figure.replace('億', '*10**8')
                    return eval(hennkan)             
                #万しかない時
                elif '万' in flow_figure and '兆' not in flow_figure and '万' not in flow_figure:
                    hennkan = flow_figure.replace('万', '*10**4')
                    return eval(hennkan)
                #兆と億の時
                elif'兆'in flow_figure and '億' in flow_figure:
                    hennkan = flow_figure.replace('兆', '*10**12+').replace('億','*10**8')
                    return eval(hennkan)
                #億と万の時
                elif'億'in flow_figure and '万' in flow_figure:
                    hennkan = flow_figure.replace('億', '*10**8+').replace('万','*10**4')
                    return eval(hennkan)
                else:
                    hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", flow_figure)
                    return eval(hennkan)
                    

In [ ]:
 for s in op_list:
                hennkan = s.split('%')[0]  # '15.65' のような文字列を取得
                zop_list.append(eval(hennkan))

In [2]:
def float_hennkan(s):
    # *だけの時
    if '*' in s:
        s = s.split('円')[0]  # '15.65' のような文字列を取得
        s = s.replace('*', '')  # * を削除
        integer_part, decimal_part =s.split('.')  # '15' と '65' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
        hennkan_result= f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result
        
        
    #小数点の時
    elif '.' in s :
        number_str = s.split('円')[0]  # '15.65' のような文字列を取得
        integer_part, decimal_part = number_str.split('.')  # '15' と '65' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
        hennkan_result = f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result
      
#     if '.' in s:
#         number_str1 = kn .split('円')[0]  # '86' のような文字列を取得
#         integer_part1 = number_str1.zfill(2)  # 整数部を 0 で埋める
#         hennkan_result = f"{integer_part1}.00"
        return hennkan_result
    #*だけの時
    elif '*' in s:
        kn = s.split('*')[1]
        number_str1 = kn .split('円')[0]  # '86' のような文字列を取得
        integer_part1 = number_str1.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part1}.00"
        return hennkan_result
    else:
        # 整数部と小数部が分離されていない場合
        number_str = s.split('円')[0]  # '86' のような文字列を取得
        integer_part = number_str.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part}.00"
        return hennkan_result
    
    
def convert_return(return_figure: str) -> str:
    if '.' in return_figure:
        number_str = return_figure.split('%')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    elif '*' in s:
        kn = s.split('*')[1]
        number_str1 = kn .split('%')[0]  # '86' のような文字列を取得
        integer_part1 = number_str1.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part1}.00"
        return hennkan_result
    else:
        integer_part = return_figure.split('%')[0]
        integer_part = integer_part.zfill(2)
        return f"{integer_part}.00"

In [9]:
def floatparcent_hennkan(s):
    if '.' in s:
        number_str = s.split('%')[0]  # '15.65' のような文字列を取得
        integer_part, decimal_part = number_str.split('.')  # '15' と '65' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
        hennkan_result = f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result

    else:
        # 整数部と小数部が分離されていない場合
        integer_part = s.split('%')[0]  # '86' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part}.00"
        return hennkan_result